## Data preprocessing and splitting

In [1]:
import os
Dataset_path_dapt_2020 = 'data_raw/dapt2020'
args_dataset_root = "data_div/dapt2020"

if not os.path.exists(args_dataset_root):
    os.makedirs(args_dataset_root)
    os.makedirs(os.path.join(args_dataset_root, 'train'))
    os.makedirs(os.path.join(args_dataset_root, 'test'))

dapt_2020_csv_name = ['enp0s3-monday.pcap_Flow.csv',
                      'enp0s3-public-tuesday.pcap_Flow.csv',
                      'enp0s3-public-wednesday.pcap_Flow.csv',
                      'enp0s3-public-thursday.pcap_Flow.csv',
                      'enp0s3-tcpdump-friday.pcap_Flow.csv',
                      'enp0s3-monday-pvt.pcap_Flow.csv',
                      'enp0s3-pvt-tuesday.pcap_Flow.csv',
                      'enp0s3-pvt-wednesday.pcap_Flow.csv',
                      'enp0s3-tcpdump-pvt-friday.pcap_Flow.csv']

dapt_2020_csv_name2 = [
    'enp0s3-pvt-thursday.pcap_Flow.csv',  # 缺少标题行
]

In [2]:
import pandas as pd

pd_list = []
for csv_name in dapt_2020_csv_name:
    csv_path = os.path.join(Dataset_path_dapt_2020, "csv", csv_name)
    pd_list.append(pd.read_csv(csv_path))
csv_path = os.path.join(Dataset_path_dapt_2020, "csv", dapt_2020_csv_name2[0])
pd_one = pd.read_csv(csv_path, header=None)
pd_one.columns = pd_list[0].columns
pd_list.append(pd_one)

XY_all = pd.concat(pd_list)
XY_all.shape # (86691, 85)

(86691, 85)

In [3]:

# X_all = X_all.drop(labels=['Flow ID', 'Src IP', 'Src Port',
#                     'Dst IP', 'Dst Port', 'Timestamp', 'Activity'], axis=1)
XY_all = XY_all.drop(labels=['Flow ID', 'Src IP', 'Src Port',
                             'Dst IP', 'Dst Port', 'Timestamp'], axis=1)

XY_all = XY_all.drop(labels=['Fwd PSH Flags',
                             'Fwd URG Flags',
                             'Bwd URG Flags',
                             'URG Flag Count',
                             'Fwd Bytes/Bulk Avg',
                             'Fwd Packet/Bulk Avg',
                             'Fwd Bulk Rate Avg',
                             'Bwd Bytes/Bulk Avg',
                             'Bwd Packet/Bulk Avg',
                             'Bwd Bulk Rate Avg',
                             'FWD Init Win Bytes',
                             'Fwd Seg Size Min',
                             ],
                     axis=1)

replacement_mapping_dict = {'Benign': 0, 'BENIGN': 0, 'Reconnaissance': 1,
                            'Establish Foothold': 2, 'Lateral Movement': 3, 'Data Exfiltration': 4}
XY_all['Stage'] = XY_all['Stage'].map(replacement_mapping_dict)
XY_all.head()


,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Activity,Stage
0,6,4308182,11,21,728.0,408.0,192.0,0.0,66.181818,54.972390,...,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,Normal,0
1,6,102319,3,5,87.0,148.0,87.0,0.0,29.000000,50.229473,...,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,Normal,0
2,17,119674949,89,1,25803.0,289.0,296.0,288.0,289.921348,2.417854,...,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,Normal,0
3,0,119999065,248,1,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000e+00,0.0,0.0,0.0,0.000000,0.0,0.0,Normal,0
4,6,110289343,19,16,1184.0,2216.0,296.0,0.0,62.315789,123.980757,...,840248.3,1.420618e+06,3937585.0,15535.0,10187129.3,67317.195083,10224606.0,10025811.0,Normal,0


In [4]:
from sklearn.model_selection import train_test_split

XY_all_0123 = XY_all.loc[XY_all['Stage'] != 4]
XY_all_4 = XY_all.loc[XY_all['Stage'] == 4]

Y_all_0123 = XY_all_0123.pop('Stage')
X_train, X_test, y_train, y_test = train_test_split(XY_all_0123, Y_all_0123, train_size = 0.7, stratify=Y_all_0123, random_state = 0) #shuffle=False
pd.Series(y_train).value_counts(), pd.Series(y_test).value_counts() 

(0    44598
 1     8336
 2     6023
 3     1716
 Name: Stage, dtype: int64,
 0    19114
 1     3573
 2     2581
 3      735
 Name: Stage, dtype: int64)

## train clinet data generation

In [5]:
X_train.loc[X_train['Activity'] == 'BENIGN', 'Activity'] = 'Normal' # BENIGN Normal 统称 Normal
Activity_counts = X_train['Activity'].value_counts()
Activity_counts

Normal                    44598
Directory Bruteforce       6985
Network Scan               5429
Web Vulnerability Scan     1781
Account Discovery          1683
Account Bruteforce          107
SQL Injection                55
Backdoor                     15
Privilege Escalation          8
Command Injection             8
CSRF                          3
Malware Download              1
Name: Activity, dtype: int64

In [6]:
index_all = list(Activity_counts.index)
for index in Activity_counts.index:
    if(Activity_counts[index]<=10):
        index_all.remove(index)
index_all



['Normal',
 'Directory Bruteforce',
 'Network Scan',
 'Web Vulnerability Scan',
 'Account Discovery',
 'Account Bruteforce',
 'SQL Injection',
 'Backdoor']

In [7]:
import random
for clint_num in range(400):
    clint_content = pd.DataFrame()

    indexsample = random.sample(index_all, 5)
    for index in indexsample:
        index_content = X_train.loc[X_train['Activity'] == index]
        index_content_sample = index_content.sample(15, replace=True)  # 每次运行sample的是不一样的
        clint_content = clint_content.append(index_content_sample)

    path = os.path.join(args_dataset_root, "train", str(clint_num)+".csv")
    clint_content.to_csv(path)

## test client data generation

In [8]:
XY_test_0123 = pd.concat([X_test, y_test], axis=1)
XY_test = pd.concat([XY_test_0123, XY_all_4])
XY_test = XY_test.drop(labels=['Activity'], axis=1)

Stage_counts = XY_test['Stage'].value_counts()
Stage_counts

0    19114
1     3573
2     2581
3      735
4       15
Name: Stage, dtype: int64

In [9]:
sample_number=[1235, 935, 835, 735, 15]
for clint_num in range(400,410):
    clint_content = pd.DataFrame()
    
    for index in range(5):
        index_content = XY_test.loc[XY_test['Stage'] == index]
        index_content_sample = index_content.sample(sample_number[index])
        clint_content = clint_content.append(index_content_sample)

    path = os.path.join(args_dataset_root, "test", str(clint_num)+".csv")
    clint_content.to_csv(path)